In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import roc_auc_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/test.csv')
sample_soln_df = pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/sample_solution.csv')

In [ ]:
sample_soln_df.head()

In [ ]:
train.shape
train.head()

In [ ]:
test.shape

In [ ]:
train.claim.value_counts()

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
# Getting the features and target column:
FEATURES = train.columns[:-1]
TARGET = train.columns[-1]

In [ ]:
# Filling in the missing values with avg value of the column:
for col in FEATURES:
    avg_val = train[col].mean()
    train[col].fillna(avg_val, inplace=True)
    test[col].fillna(avg_val, inplace=True)

In [ ]:
test_preds = 0
train_oof = np.zeros((train.shape[0],1))
train_oof = pd.DataFrame(train_oof, columns=['preds'])
kf = KFold(n_splits=5)
for train_index, test_index in kf.split(train[FEATURES]):
    clf = HistGradientBoostingClassifier()
    trn_X, trn_y = train.loc[train_index, FEATURES], train.loc[train_index, TARGET]
    val_X, val_y = train.loc[test_index, FEATURES], train.loc[test_index, TARGET]
    clf.fit(trn_X,trn_y)
    oof_proba = clf.predict_proba(val_X)[:,1]
    train_oof.loc[test_index,'preds'] = oof_proba
    tmp = clf.predict_proba(test[FEATURES])
    test_preds += tmp/5
    
print("The score for OOF is:")
roc_auc_score(train['claim'], train_oof['preds'])

In [ ]:
sample_soln_df['claim'] = test_preds[:,1]
sample_soln_df.to_csv('submission.csv', index=False)

this will be approchable but am i making my model slow ? 
as its going to iterate for this big data set.